In [ ]:
import pandas as pd

from route_66.visualisation import plot_simulation, plot_batch_run

# Simulational Analysis
Jupyter Notebook for the assignment 'Touring Machines' for the course Simulation Tooling for the bachelor programme Artificial Intelligence at the HU University of Applied Sciences Utrecht

In [ ]:
plot_simulation(100, 35, length=100, p=0.3, grid=True)

## Analysis
### Plot
Plot containing boxplots visualising the effects of timer length and initial number of cars on average velocity:

In [ ]:
data = plot_batch_run(N=[10, 25, 35], timer=[0, 1, 2, 4, 6], iterations=20)

### Using Pandas


In [ ]:
# Drop Run, length and lanes, and group Dataframe by initial cars on road 
N_data = data[["timer", "N", "Average Velocity", "Standard Deviation"]].groupby("N")

# For each group, split from dataframe, group by timer, find mean for velocity and STD, sort by Average velocity
timer_data = [N_data.get_group(x).groupby("timer").mean().sort_values("Average Velocity", ascending=False) for x in N_data.groups]
print(timer_data[0])

In [ ]:
mean_data = pd.concat(timer_data)
mean_data

For each variation of N, the timer with a length of 6 grants the greatest average average velocity. It usually also grants a lower or the lowest average standard deviation for its N.   

The greater the amount of initial cars, the lesser the impact of timer length. 